In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from simpletransformers.classification import ClassificationModel, ClassificationArgs

ModuleNotFoundError: No module named 'pandas'

In [5]:
import pandas as pd
data = pd.read_json('foxnews_news.json')
print(data.sentiment.value_counts())
print(data[data.sentiment == ''])

neutral     236
negative     98
positive     76
NULL          2
Name: sentiment, dtype: int64
Empty DataFrame
Columns: [url, body, sentiment, authorUrl, authorName, author_url, author_name]
Index: []


In [3]:
print(torch.__version__)
print(torch.cuda.is_available())

1.8.0
True


In [ ]:
df = pd.read_json('foxnews_news.json')
df.head()

In [4]:
datapath = 'data/'
train_df = pd.read_csv(datapath + 'train.csv', header=None)
train_df.head()

,0,1
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [5]:
test_df = pd.read_csv(datapath + 'test.csv', header=None)
test_df.head()

,0,1
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...


In [6]:
train_df[0] = (train_df[0] == 2).astype(int)
test_df[0] = (test_df[0] == 2).astype(int)

In [7]:
train_df = pd.DataFrame({"text": train_df[1].replace(r"\n", " ", regex=True), "labels": train_df[0]})
print(train_df.head())

                                                text  labels
0  Unfortunately, the frustration of being Dr. Go...       0
1  Been going to Dr. Goldberg for over 10 years. ...       1
2  I don't know what Dr. Goldberg was like before...       0
3  I'm writing this review to give you a heads up...       0
4  All the food is great here. But the best thing...       1


In [8]:
test_df = pd.DataFrame({"text": test_df[1].replace(r"\n", " ", regex=True), "labels": test_df[0]})
print(test_df.head())

                                                text  labels
0  Contrary to other reviews, I have zero complai...       1
1  Last summer I had an appointment to get new ti...       0
2  Friendly staff, same starbucks fair you get an...       1
3  The food is good. Unfortunately the service is...       0
4  Even when we didn't have a car Filene's Baseme...       1


In [9]:
model_name = "bert-base-cased"
model_type = "bert"

model_args = ClassificationArgs()
#model_args.no_cache = True
model_args.fp16 = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.output_dir = f"outputs/{model_type}"
model_args.best_model_dir = f"outputs/{model_type}/best_model"
model_args.evaluate_during_training = True
model_args.max_seq_length = 128
model_args.num_train_epochs = 3
model_args.train_batch_size = 128
model_args.eval_batch_size = 64

In [ ]:
#test_df.text.map(lambda x: len(x.split(' '))).max()

In [14]:
model = ClassificationModel(model_type, model_name, args=model_args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [15]:
model.train_model(train_df[:int(len(test_df)* 0.2)], eval_df=test_df[:int(len(test_df)* 0.1)])

  0%|          | 0/7600 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/3800 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/3800 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/3800 [00:00<?, ?it/s]

(180,
 {'global_step': [60, 120, 180],
  'tp': [1535, 1614, 1545],
  'tn': [1922, 1839, 1946],
  'fp': [159, 242, 135],
  'fn': [184, 105, 174],
  'mcc': [0.8176619224051656, 0.8191061893049382, 0.8357345383821015],
  'train_loss': [0.25530534982681274, 0.09243131428956985, 0.0394628681242466],
  'eval_loss': [0.22428959545989832, 0.23101878501474857, 0.24265223648399115],
  'auroc': [0.9705880708557633, 0.975400022195889, 0.9736864101056709],
  'auprc': [0.9670278416090311, 0.9725475330743457, 0.9686972305057919]})

In [16]:
preds, out = model.predict(["Contrary to other reviews, I have zero complaints about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and know what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. But here, my service and road coverage has all been well explained - and let up to me"])
print(preds)
print(out)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[1]
[[0.07775879 0.3815918 ]]


In [ ]:
train_df[:int(len(test_df)* 0.1)]